Given a vector $v\in \{-1,+1\}^n$, we are interested in the set of $k$ orthogonal vectors $r_1,\ldots, r_k \in \{-1,+1\}^n$, that solves
$$\max_{\{r_i\}_k}\min_{i\in [k]}v^Tr_i$$

From Tramer et al., 2018, it was shown that if for all $i$, $v^Tr_i \geq \alpha  n$ for $\alpha \in (0,1)$, then $\alpha \leq 1/\sqrt{k}$. In other words, $OPT=n/\sqrt{k}$. Let's denote by $R$ the matrix formed by stacking $r_i$ vertically.

In [58]:
from scipy.linalg import hadamard
import numpy as np
import itertools

In [59]:
# dim
n = 108 # imagenet dim
# adv cone size < n
k = 36
# target vector
v = np.sign(np.random.randn(n))
OPT= n / np.sqrt(k)

#### Naive Construction

The following naive method (Tramer et al, 2018) can be used to achieve ~$n/k$, a factor of $\sqrt{k}$ worse than OPT, assuming $k$ divides $n$.

In [60]:
def chain_lol(lol):
    return list(itertools.chain(*lol))

def construct_idx(chunk_size, k, n):
    """a method to get 1d idxs for the R matrix used by the naive construction method
    """
    return list(filter(
        lambda x: x < n*k,
        chain_lol(
        [range(i*(chunk_size)+j,i*(chunk_size ) + chunk_size+j) for i,j in enumerate(range(0,n*k,n))])))

def naive_R(n, k):
    chunk_size = (n + k - 1) // k
    R = np.zeros((k, n))
    #print(construct_idx(chunk_size, k, n))
    R.ravel()[construct_idx(chunk_size, k, n)] = v
    return R

In [61]:
R = naive_R(n,k)

In [62]:
R.dot(v), OPT 

(array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
        3., 3.]), 18.0)

A factor of 10 worse for Imagenet. Let's consider another construction from Tramer et al., 2018.

#### Tight Randomized Construction with Regular Hadamard matrix

In [63]:
Hs = {
    '4': np.load('../reg_hadamard_mats/reg_hadamard_mat_order-4.npy'),
    '16':np.load('../reg_hadamard_mats/reg_hadamard_mat_order-16.npy'),
    '36':np.load('../reg_hadamard_mats/reg_hadamard_mat_order-36.npy'),
    '64':np.load('../reg_hadamard_mats/reg_hadamard_mat_order-64.npy'),
    '100':np.load('../reg_hadamard_mats/reg_hadamard_mat_order-100.npy'),
}

In [64]:
H = Hs['100']

In [65]:
k = H.shape[0]
# target vector
v = np.sign(np.random.randn(n))
OPT= n / np.sqrt(k)

In [66]:
R = np.zeros((k, n))
R[:, :n // k * k ] = np.repeat(H, n // k, axis=1)
R *= v[None, :]

In [67]:
R.dot(v), OPT 

(array([10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10.,
        10., 10., 10., 10., 10., 10., 10., 10., 10.]), 10.8)

In [68]:
sum(H)

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

In [69]:
H.dot(H.T)

array([[100,   0,   0, ...,   0,   0,   0],
       [  0, 100,   0, ...,   0,   0,   0],
       [  0,   0, 100, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 100,   0,   0],
       [  0,   0,   0, ...,   0, 100,   0],
       [  0,   0,   0, ...,   0,   0, 100]])